In [3]:
!pip install pyDrive # Descargar el archive en drive

In [4]:
!pip install tsp_solver2 # Greddy Algorithm para resolver el problema del viajero

In [6]:
!pip install pygraphviz # graphviz in python!

In [7]:
!pip install networkx  # Creacion de redes complejas en python

In [9]:
from tsp_solver.greedy import solve_tsp
from contextlib import contextmanager
import time
import pandas as pd
import numpy as np
import gc
import networkx as nx
import string

# Contola el tiempo de tus procesos.
@contextmanager
def timer(title):
  t0 = time.time()
  yield
  print("%s - done in %0.3f" % (title, time.time() - t0))

In [10]:
def rutastxt_to_df(filename):
  ''' Convierta el archivo rutas.txt a una matriz de distancias
      Argumentos:
        - filename: Nombre completo del archivo de texto
  '''
  rutas = pd.read_csv(filename,header = None,sep = '=')
  cities = {'city_A':[],'city_B':[],'distance':[]}
  
  # 1. Creamos un diccionario
  for y in rutas.index:
    ctA, ctB = [x.strip() for x in rutas[0][y].split('to')]  
    cities['city_A'].append(ctA)
    cities['city_B'].append(ctB)
    cities['distance'].append(rutas[1][y])

  # 2. Duplicamos el dataframe considerando que la distancia de A -> B es la misma que B <- A.
  rutas = pd.DataFrame(cities)
  rutas2 = rutas.copy()
  rutas2['city_A'] = rutas['city_B']
  rutas2['city_B'] = rutas['city_A']
  rutas3 = pd.concat([rutas,rutas2])
  
  # 3. Redimencionamos nuestro data.frame
  rutas_final = rutas3.pivot(index = 'city_A',columns = 'city_B',values = 'distance')
  rutas_final[np.isnan(rutas_final)] = 0
  
  # 4. Liberamos memoria.
  del cities, rutas,rutas2,rutas3
  gc.collect()

  return rutas_final

In [11]:
def graph_viz(dist_mtx):
  dt = [('len', float)]
  A = dist_mtx.values/10
  A = A.view(dt)

  G = nx.from_numpy_matrix(A)
  G = nx.relabel_nodes(G, dict(zip(range(len(G.nodes())),string.ascii_uppercase)))    

  G = nx.drawing.nx_agraph.to_agraph(G)

  G.node_attr.update(color="red", style="filled")
  G.edge_attr.update(color="blue", width="2.0")
  G.draw('out.png', format='png', prog='neato')

In [12]:
with timer('Creando la matriz de distancias'):
  mtx_dist = rutastxt_to_df('rutas.txt')

with timer('Crear una vizualizacion del grafo'):
  graph_viz(mtx_dist)
  
with timer('Resolviendo el problema del viajero (Greedy algorithm)'):
  path = solve_tsp(mtx_dist.values)

Creando la matriz de distancias - done in 0.277
Crear una vizualizacion del grafo - done in 0.210
Resolviendo el problema del viajero (Greedy algorithm) - done in 0.001


In [13]:
from matplotlib.pyplot import imread, imshow
#imshow(imread('out.png'))

In [14]:
mtx_dist

city_B,AlphaCentauri,Arbre,Faerun,Norrath,Snowdin,Straylight,Tambi,Tristram
city_A,,,,,,,,
AlphaCentauri,0.0,108.0,60.0,34.0,66.0,34.0,28.0,3.0
Arbre,108.0,0.0,83.0,60.0,71.0,81.0,40.0,90.0
Faerun,60.0,83.0,0.0,63.0,12.0,21.0,39.0,57.0
Norrath,34.0,60.0,63.0,0.0,91.0,9.0,113.0,50.0
Snowdin,66.0,71.0,12.0,91.0,0.0,121.0,22.0,111.0
Straylight,34.0,81.0,21.0,9.0,121.0,0.0,130.0,27.0
Tambi,28.0,40.0,39.0,113.0,22.0,130.0,0.0,35.0
Tristram,3.0,90.0,57.0,50.0,111.0,27.0,35.0,0.0


In [15]:
# suma de caminos
min_distance = sum([mtx_dist.iloc[x] for x in list(zip(path,path[1:]))])
print(' -> '.join(list(mtx_dist.index[path])) + ' = ' + str(min_distance))

Arbre -> Tambi -> Snowdin -> Faerun -> Straylight -> Norrath -> AlphaCentauri -> Tristram = 141.0
